In [1]:
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score
import json
from sklearn.model_selection import train_test_split
from collections import Counter
from itertools import combinations

In [2]:
train = pd.read_csv('../data/train.csv')
label = pd.read_csv('../data/train_label.csv')
test = pd.read_csv('../data/test.csv')
submission = pd.read_csv('../data/submission.csv')
train_data = pd.merge(train,label,on='ID')



/Users/zhaoliang/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
train_data.head()

,ID,企业类型,经营期限至,登记机关,企业状态,邮政编码,投资总额,注册资本,核准日期,行业代码,...,一年内到期的非流动资产_年末数,油气资产_年末数,应收补贴款_年末数,应收股利_年末数,应收利息_年末数,应收票据_年末数,预付款项_年末数,资产总计_年末数,在建工程_年末数,Label
0,1,9.0,NaN,1.0,0.0,266300,19.110000,19.110000,00:00.0,5154.0,...,0.0,0.0,0.0,0.0,0.0,28.122902,1.004419,632.032182,0.0,0
1,3,11.0,NaN,5.0,0.0,266000,29.154194,19.210446,00:00.0,7214.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,9,0.0,NaN,3.0,0.0,266000,NaN,19.120045,13:31.0,2110.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,10,0.0,42:05.0,9.0,0.0,266000,NaN,19.160223,00:00.0,5163.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,14,0.0,NaN,2.0,0.0,266400,NaN,19.210446,00:00.0,5439.0,...,0.0,0.0,0.0,0.0,0.0,10.044194,0.508236,319.205293,0.0,0


In [4]:
def deal_time(x):
    if str(x) == 'nan':
        return x
    
    x = str(x)
    parts = x.split(':')
    return int(parts[0]) + float(parts[1])/100

In [5]:
def cal_diff(x, y):
    if str(x) == 'nan':
        return x
    elif str(y) == 'nan':
        return y
    else:
        return y-x

In [6]:
def deal_manage_range(x):
    g = json.loads(x)
    return len(g)

In [7]:
def deal_cat(x):
    if str(x) == 'nan':
        return -1
    return int(x)

In [8]:
def get_built_tax_rate(x, y, z):  #x:cj y:zz z:xf
    if str(x) == 'nan':
        return x
    elif str(y) == 'nan':
        return y
    elif str(z) == 'nan':
        return z
    elif y+z==0:
        return 0
    else:
        return x/(y+z)

In [9]:
def get_xf_tax(x, y): #消费税 x:jy  y:zz
    if str(x) == 'nan':
        return x
    elif str(y) == 'nan':
        return y
    return x/0.03 - y

In [10]:
def get_tax(x, y): #y->tax
    if str(x) == 'nan':
        return x
    elif str(y) == 'nan':
        return y
    elif y == 0:
        return 0
    return x/y

In [11]:
all_ranges = []
jyfw = list(train_data['经营范围'])+list(test['经营范围'])
for item in jyfw:
    all_ranges += json.loads(item)
range_dict = dict(Counter(all_ranges))
# bins = [4,8,16,64,128,256,600,3000]
def deal_manage_range_bin(x, lower, upper=100000):
    count = 0
    items = json.loads(x)
    for item in items:
        counts = range_dict.get(item)
        if counts and counts<=upper and counts>lower:
            count += 1
    return count

In [12]:
industry_code_arr = []
industry_code_list = list(train['行业代码'].fillna(-1))+list(test['行业代码'].fillna(-1))
for code in industry_code_list:
    industry_code_arr.append(code)
    
industry_code_map = dict(Counter(industry_code_arr))
def deal_industry_code(x):
    if industry_code_map.get(x) == 4644:
        return 1
    elif industry_code_map.get(x) == 1993:
        return 2
    elif industry_code_map.get(x) == 1272:
        return 0
    elif industry_code_map.get(x) in range(259,1272):
        return 3
    elif industry_code_map.get(x) in range(100,259):
        return 4
    elif industry_code_map.get(x) in range(50,100):
        return 5
    elif industry_code_map.get(x) in range(0,50):
        return 6
    else:
        return 7

In [13]:
def deal_industry_class(x):
    if x>=12:
        return 12
    return x

In [14]:
#企业类型

industry_type_arr = []
industry_type_list = list(train['企业类型'].fillna(-1))+list(test['企业类型'].fillna(-1))
for type_ in industry_type_list:
    industry_type_arr.append(type_)
    
industry_type_map = dict(Counter(industry_type_arr))
def deal_industry_type(x):
    if industry_type_map.get(x)==9311:
        return 1
    elif industry_type_map.get(x)==4406:
        return 2
    elif industry_type_map.get(x)==2750:
        return 3
    elif industry_type_map.get(x)==1262:
        return 4
    elif industry_type_map.get(x) in range(200,1262):
        return 5
    elif industry_type_map.get(x) in range(30, 200):
        return 6
    elif industry_type_map.get(x)<30:
        return 7
    else:
        return 8

In [15]:
def is_changed(x, y):
    if str(x) == 'nan':
        return x
    elif str(y) == 'nan':
        return y
    elif x == y:
        return 1
    else:
        return 0

In [17]:
def enus2classfify(x):
    if str(x) == 'nan':
        return -1
    else:
        return 0

In [18]:
train_data['经营期限至'] = train_data['经营期限至'].apply(deal_time)
train_data['经营期限自'] = train_data['经营期限自'].apply(deal_time)
train_data['成立日期'] = train_data['成立日期'].apply(deal_time)
train_data['核准日期'] = train_data['核准日期'].apply(deal_time)
train_data['注销时间'] = train_data['注销时间'].apply(deal_time)

train_data['经营期限'] = train_data.apply(lambda row: cal_diff(row['经营期限自'], row['经营期限至']), axis=1)
train_data['changed'] = train_data.apply(lambda row: is_changed(row['核准日期'], row['成立日期']), axis=1)

train_data['消费税'] = train_data.apply(lambda row: get_xf_tax(row['教育费'], row['增值税']), axis=1)
train_data['城建税率'] = train_data.apply(lambda row: get_built_tax_rate(row['城建税'], row['增值税'], row['消费税']), axis=1)
train_data['应纳税额'] = train_data.apply(lambda row: get_tax(row['企业所得税'], row['城建税率']), axis=1)

train_data['经营范围1'] = train_data['经营范围'].apply(deal_manage_range_bin, args=(0,4,))
train_data['经营范围2'] = train_data['经营范围'].apply(deal_manage_range_bin, args=(4,8,))
train_data['经营范围3'] = train_data['经营范围'].apply(deal_manage_range_bin, args=(8,16,))
train_data['经营范围4'] = train_data['经营范围'].apply(deal_manage_range_bin, args=(16,64,))
train_data['经营范围5'] = train_data['经营范围'].apply(deal_manage_range_bin, args=(64,128,))
train_data['经营范围6'] = train_data['经营范围'].apply(deal_manage_range_bin, args=(128,200,))
train_data['经营范围7'] = train_data['经营范围'].apply(deal_manage_range_bin, args=(200,1000,))
train_data['经营范围8'] = train_data['经营范围'].apply(deal_manage_range_bin, args=(1000,5000,))
train_data['经营范围9'] = train_data['经营范围'].apply(deal_manage_range_bin, args=(5000,))


train_data['经营范围'] = train_data['经营范围'].apply(deal_manage_range)

cat_feat = ['行业代码','行业门类','管辖机关', '企业类别','企业类型','登记机关']
for item in cat_feat:
    train_data[item] = train_data[item].apply(deal_cat)
    
train_data['行业代码plus'] = train_data['行业代码'].apply(deal_industry_code)

train_data['行业门类plus'] = train_data['行业门类'].apply(deal_industry_class)

train_data['企业类型plus'] = train_data['企业类型'].apply(deal_industry_type)

c2c = ['长期负债合计_年初数','长期负债合计_年末数','其他负债（或长期负债）_年初数','其他负债（或长期负债）_年末数','其他应交款_年初数',
'其他应交款_年末数','应付福利费_年初数','应付福利费_年末数','预提费用_年初数','预提费用_年末数','待摊费用_年初数','待摊费用_年末数',
'应收补贴款_年初数','应收补贴款_年末数','长期投资合计_年初数','长期投资合计_年末数','固定资产净额_年初数','固定资产净额_年末数',
'固定资产净值_年初数','固定资产净值_年末数','无形资产及其他资产合计_年初数','无形资产及其他资产合计_年末数']

columns = train_data.columns.tolist()
columns = list(set(columns)- set(c2c))
for item in columns:
    parts = item.split("_")
    if len(parts) == 2:
        if parts[1] == '年初数':
            train_data[parts[0] + '_diff'] = train_data.apply(lambda row: cal_diff(row[item], row[parts[0]+"_年末数"]), axis=1)

for item in c2c:
    train_data[item] = train_data[item].apply(enus2classfify)
    
cat_feats=cat_feat+['行业代码plus','行业门类plus','企业类型plus']+c2c

feats = train_data.columns

#连续变量与连续变量的组合        
# picked_con=['城建税', '未分配利润_年末数', '教育费', '货币资金_年初数', '印花税', '增值税', '企业所得税']
# for zuhe in combinations(picked_con,2):
#     name_add=zuhe[0]+'+'+zuhe[1]
#     name_sub=zuhe[0]+'-'+zuhe[1]
#     name_mul=zuhe[0]+'*'+zuhe[1]
#     name_div=zuhe[0]+'/'+zuhe[1]
#     train_data[name_add]=list(map(lambda x,y:x+y,train_data[zuhe[0]].values,train_data[zuhe[1]].values))
#     train_data[name_sub]=list(map(lambda x,y:x-y,train_data[zuhe[0]].values,train_data[zuhe[1]].values))
#     train_data[name_mul]=list(map(lambda x,y:x*y,train_data[zuhe[0]].values,train_data[zuhe[1]].values))
#     train_data[name_div]=list(map(lambda x,y:x/y if y!=0 else 0,train_data[zuhe[0]].values,train_data[zuhe[1]].values))

In [19]:
drop_feat = ['ID','邮政编码','Label']
feat = [item for item in feats if item not in drop_feat]
# cat_feat = ['管辖机关', '企业类别','行业门类','行业代码','企业状态','邮政编码','登记机关','企业类型']
X = train_data[feat]
y = train_data['Label'].values
x_train,x_test,y_train,y_test = train_test_split(X, y, test_size=0.3,random_state=42)

In [20]:
def make_classifier():
    clf = CatBoostClassifier(
                               loss_function='Logloss',
                               eval_metric="AUC",
                               task_type="CPU",
                               learning_rate=0.01,
                               iterations=20000,
                               od_type="Iter",
        
#                                depth=8,
                               early_stopping_rounds=500,
    #                            l2_leaf_reg=1,
    #                            border_count=96,
                               random_seed=2019
                              )
        
    return clf



In [21]:
cat_clf = make_classifier()
cat_clf.fit(x_train, y_train, eval_set=(x_test, y_test),
            use_best_model=True, verbose=500,cat_features=cat_feats)

0:	test: 0.8725364	best: 0.8725364 (0)	total: 92.3ms	remaining: 30m 46s
500:	test: 0.9248437	best: 0.9249182 (485)	total: 12.3s	remaining: 8m
1000:	test: 0.9262648	best: 0.9264718 (968)	total: 24.8s	remaining: 7m 50s
1500:	test: 0.9281105	best: 0.9281976 (1470)	total: 37.3s	remaining: 7m 39s
2000:	test: 0.9293165	best: 0.9293407 (1992)	total: 50s	remaining: 7m 30s
2500:	test: 0.9295641	best: 0.9295873 (2476)	total: 1m 2s	remaining: 7m 19s
3000:	test: 0.9302797	best: 0.9303349 (2966)	total: 1m 15s	remaining: 7m 8s
3500:	test: 0.9303194	best: 0.9306240 (3220)	total: 1m 28s	remaining: 6m 58s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.9306240178
bestIteration = 3220

Shrink model to first 3221 iterations.


In [22]:
pd.set_option('max_rows', None)
feature_importance_df = pd.DataFrame()
feature_importance_df["Feature"] = feat
feature_importance_df["importance"] = [item.round(2) for item in cat_clf.feature_importances_]
feature_importance_df.sort_values('importance')

,Feature,importance
123,无形资产及其他资产合计_年初数,0.00
149,固定资产净额_年末数,0.00
31,其他非流动负债_年初数,0.00
151,固定资产清理_年末数,0.00
153,交易性金融资产_年末数,0.00
26,非流动负债合计_年初数,0.00
144,待摊费用_年末数,0.00
137,存货中的周转材料_年末数,0.00
55,专项应付款_年初数,0.00
57,长期负债合计_年末数,0.00


In [23]:
prob_oof = cat_clf.predict_proba(x_test)
prob_oof = [item[1] for item in prob_oof]
score = roc_auc_score(y_test, prob_oof)
print(score)

prob_oof = cat_clf.predict_proba(x_train)
prob_oof = [item[1] for item in prob_oof]
score = roc_auc_score(y_train, prob_oof)
print(score)

0.930624017794541
0.9775441751635858


In [28]:
def feat_filter(feature_importance_df, cat_feats, thred = 0.01):
    arr = []
    while((feature_importance_df['importance']<thred).any()):
        filted_feat = list(feature_importance_df[feature_importance_df['importance']>thred]['Feature'])
        X = train_data[filted_feat]
        y = train_data['Label'].values
        x_train,x_test,y_train,y_test = train_test_split(X, y, test_size=0.3,random_state=42)

        cat_clf = make_classifier()
        cat_feats = list(set(cat_feats)&set(filted_feat))
        cat_clf.fit(x_train, y_train, eval_set=(x_test, y_test),
        use_best_model=True, verbose=0,cat_features=cat_feats)

        prob_oof = cat_clf.predict_proba(x_test)
        prob_oof = [item[1] for item in prob_oof]
        test_score = roc_auc_score(y_test, prob_oof)


        prob_oof = cat_clf.predict_proba(x_train)
        prob_oof = [item[1] for item in prob_oof]
        train_score = roc_auc_score(y_train, prob_oof)

        print('round:')
        print('train_auc:',train_score)
        print('test_auc:',test_score)
        print('-----------------------------------')
        pd.set_option('max_rows', None)
        feature_importance_df = pd.DataFrame()
        feature_importance_df["Feature"] = filted_feat
        feature_importance_df["importance"] = [item.round(2) for item in cat_clf.feature_importances_]
        
        arr.append(filted_feat)
    return arr

In [29]:
arr = feat_filter(feature_importance_df, cat_feats, 0.001)

round:
train_auc: 0.9774199495318786
test_auc: 0.9298435724474747
-----------------------------------
round:
train_auc: 0.9747408141484982
test_auc: 0.9295423224776963
-----------------------------------
round:
train_auc: 0.9766557713190477
test_auc: 0.9307821377626266
-----------------------------------
round:
train_auc: 0.9774529311648428
test_auc: 0.9312110442204008
-----------------------------------
round:
train_auc: 0.982150635829877
test_auc: 0.9305137690094534
-----------------------------------
round:
train_auc: 0.9849319831606983
test_auc: 0.9327467904547762
-----------------------------------
round:
train_auc: 0.9810202370321508
test_auc: 0.9309228500278041
-----------------------------------
round:
train_auc: 0.9794369630761284
test_auc: 0.9331839172167017
-----------------------------------
round:
train_auc: 0.977738097830991
test_auc: 0.9315548463528446
-----------------------------------
round:
train_auc: 0.981967369963689
test_auc: 0.9317226372669907
-------------------

In [31]:
filted_feats = arr[7]
X = train_data[filted_feats]
y = train_data['Label'].values
x_train,x_test,y_train,y_test = train_test_split(X, y, test_size=0.3,random_state=42)

cat_clf = make_classifier()
cat_feats = list(set(cat_feats)&set(filted_feats))
cat_clf.fit(x_train, y_train, eval_set=(x_test, y_test),
use_best_model=True, verbose=0,cat_features=cat_feats)

prob_oof = cat_clf.predict_proba(x_test)
prob_oof = [item[1] for item in prob_oof]
test_score = roc_auc_score(y_test, prob_oof)


prob_oof = cat_clf.predict_proba(x_train)
prob_oof = [item[1] for item in prob_oof]
train_score = roc_auc_score(y_train, prob_oof)

In [ ]:
round:
train_auc: 0.9778491774816346
test_auc: 0.9313706148303956
-----------------------------------
round:
train_auc: 0.978490452439552
test_auc: 0.9326674886970817
-----------------------------------
round:
train_auc: 0.984600144372431
test_auc: 0.9330228959647977
-----------------------------------
round:
train_auc: 0.9840691711964553
test_auc: 0.9325673944053577
-----------------------------------
round:
train_auc: 0.9832174827546509
test_auc: 0.93267522545393
-----------------------------------
round:
train_auc: 0.9836698134048558
test_auc: 0.933713885060806
-----------------------------------
round:
train_auc: 0.9773896126525011
test_auc: 0.9308242063779889
-----------------------------------

In [ ]:
filted_feat = list(feature_importance_df[feature_importance_df['importance']>0.01]['Feature'])
X = train_data[filted_feat]
y = train_data['Label'].values
x_train,x_test,y_train,y_test = train_test_split(X, y, test_size=0.3,random_state=42)

In [40]:
cat_clf = make_classifier()
cat_clf.fit(x_train, y_train, eval_set=(x_test, y_test),
            use_best_model=True, verbose=500,cat_features=cat_feats)

0:	test: 0.8829226	best: 0.8829226 (0)	total: 33.1ms	remaining: 11m 1s
500:	test: 0.9253766	best: 0.9253766 (500)	total: 14.1s	remaining: 9m 9s
1000:	test: 0.9272160	best: 0.9273330 (998)	total: 28.8s	remaining: 9m 6s
1500:	test: 0.9290037	best: 0.9290037 (1500)	total: 43.4s	remaining: 8m 55s
2000:	test: 0.9307135	best: 0.9307135 (2000)	total: 57.6s	remaining: 8m 38s
2500:	test: 0.9310906	best: 0.9311709 (2333)	total: 1m 11s	remaining: 8m 23s
3000:	test: 0.9316322	best: 0.9316322 (3000)	total: 1m 26s	remaining: 8m 11s
3500:	test: 0.9321409	best: 0.9321428 (3454)	total: 1m 41s	remaining: 7m 58s
4000:	test: 0.9324717	best: 0.9326022 (3890)	total: 1m 56s	remaining: 7m 45s
4500:	test: 0.9325500	best: 0.9327453 (4168)	total: 2m 10s	remaining: 7m 30s
5000:	test: 0.9329784	best: 0.9330229 (4919)	total: 2m 25s	remaining: 7m 17s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.933022896
bestIteration = 4919

Shrink model to first 4920 iterations.


In [41]:
prob_oof = cat_clf.predict_proba(x_test)
prob_oof = [item[1] for item in prob_oof]
score = roc_auc_score(y_test, prob_oof)
score

0.9330228959647977

In [42]:
prob_oof = cat_clf.predict_proba(x_train)
prob_oof = [item[1] for item in prob_oof]
score = roc_auc_score(y_train, prob_oof)
score

0.984600144372431

In [43]:
pd.set_option('max_rows', None)
feature_importance_df = pd.DataFrame()
feature_importance_df["Feature"] = filted_feat
feature_importance_df["importance"] = [item.round(2) for item in cat_clf.feature_importances_]
feature_importance_df.sort_values('importance')

,Feature,importance
65,持有至到期投资_年初数,0.00
70,开发支出_年初数,0.00
36,应付债券_年初数,0.00
149,长期应收款_diff,0.01
161,开发支出_diff,0.01
97,应收股利_年末数,0.01
154,工程物资_diff,0.02
76,一年内到期的非流动资产_年初数,0.02
167,生产性生物资产_diff,0.02
79,应收票据_年初数,0.02


In [22]:
0.9299755808611977
dropit 0.9304726674886971
ascat 0.9312033074635525

SyntaxError: invalid syntax (<ipython-input-22-5e90baa79963>, line 2)

In [34]:
test['经营期限至'] = test['经营期限至'].apply(deal_time)
test['经营期限自'] = test['经营期限自'].apply(deal_time)
test['成立日期'] = test['成立日期'].apply(deal_time)
test['核准日期'] = test['核准日期'].apply(deal_time)
test['注销时间'] = test['注销时间'].apply(deal_time)

test['经营期限'] = test.apply(lambda row: cal_diff(row['经营期限自'], row['经营期限至']), axis=1)
test['changed'] = test.apply(lambda row: is_changed(row['核准日期'], row['成立日期']), axis=1)

test['消费税'] = test.apply(lambda row: get_xf_tax(row['教育费'], row['增值税']), axis=1)
test['城建税率'] = test.apply(lambda row: get_built_tax_rate(row['城建税'], row['增值税'], row['消费税']), axis=1)
test['应纳税额'] = test.apply(lambda row: get_tax(row['企业所得税'], row['城建税率']), axis=1)

test['经营范围1'] = test['经营范围'].apply(deal_manage_range_bin, args=(0,4,))
test['经营范围2'] = test['经营范围'].apply(deal_manage_range_bin, args=(4,8,))
test['经营范围3'] = test['经营范围'].apply(deal_manage_range_bin, args=(8,16,))
test['经营范围4'] = test['经营范围'].apply(deal_manage_range_bin, args=(16,64,))
test['经营范围5'] = test['经营范围'].apply(deal_manage_range_bin, args=(64,128,))
test['经营范围6'] = test['经营范围'].apply(deal_manage_range_bin, args=(128,256,))
test['经营范围7'] = test['经营范围'].apply(deal_manage_range_bin, args=(256,600,))
test['经营范围8'] = test['经营范围'].apply(deal_manage_range_bin, args=(600,3000,))
test['经营范围9'] = test['经营范围'].apply(deal_manage_range_bin, args=(3000,))

test['经营范围'] = test['经营范围'].apply(deal_manage_range)

cat_feat = ['行业代码','行业门类','管辖机关', '企业类别','企业类型','登记机关']
for item in cat_feat:
    test[item] = test[item].apply(deal_cat)
    
test['行业代码plus'] = test['行业代码'].apply(deal_industry_code)

test['行业门类plus'] = test['行业门类'].apply(deal_industry_class)

test['企业类型plus'] = test['企业类型'].apply(deal_industry_type)

c2c = ['长期负债合计_年初数','长期负债合计_年末数','其他负债（或长期负债）_年初数','其他负债（或长期负债）_年末数','其他应交款_年初数',
'其他应交款_年末数','应付福利费_年初数','应付福利费_年末数','预提费用_年初数','预提费用_年末数','待摊费用_年初数','待摊费用_年末数',
'应收补贴款_年初数','应收补贴款_年末数','长期投资合计_年初数','长期投资合计_年末数','固定资产净额_年初数','固定资产净额_年末数',
'固定资产净值_年初数','固定资产净值_年末数','无形资产及其他资产合计_年初数','无形资产及其他资产合计_年末数']

columns = test.columns.tolist()
columns = list(set(columns)- set(c2c))
for item in columns:
    parts = item.split("_")
    if len(parts) == 2:
        if parts[1] == '年初数':
            test[parts[0] + '_diff'] = test.apply(lambda row: cal_diff(row[item], row[parts[0]+"_年末数"]), axis=1)

for item in c2c:
    test[item] = test[item].apply(enus2classfify)
    



---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<ipython-input-19-29272e92c9bb> in <module>()
      9 test['应纳税额'] = test.apply(lambda row: get_tax(row['企业所得税'], row['城建税率']), axis=1)
     10 
---> 11 test['经营范围1'] = test['经营范围'].apply(deal_manage_range_bin, args=(0,4,))
     12 test['经营范围2'] = test['经营范围'].apply(deal_manage_range_bin, args=(4,8,))
     13 test['经营范围3'] = test['经营范围'].apply(deal_manage_range_bin, args=(8,16,))

~/anaconda3/lib/python3.6/site-packages/pandas/core/series.py in apply(self, func, convert_dtype, args, **kwds)
   4043             else:
   4044                 values = self.astype(object).values
-> 4045                 mapped = lib.map_infer(values, f, convert=convert_dtype)
   4046 
   4047         if len(mapped) and isinstance(mapped[0], Series):

pandas/_libs/lib.pyx in pandas._libs.lib.map_infer()

~/anaconda3/lib/python3.6/site-packages/pandas/core/series.py in f(x)
   4029 
   4030             def f(x):
-> 4031                 return func(x, *args, **kwds)
   4032 
   4033         else:

<ipython-input-11-31168e103d35> in deal_manage_range_bin(x, lower, upper)
     10     for item in items:
     11         counts = range_dict.get(item)
---> 12         if counts<=upper and counts>lower:
     13             count += 1
     14     return count

TypeError: '<=' not supported between instances of 'NoneType' and 'int'

In [35]:
test_X = test[filted_feats]
result = cat_clf.predict_proba(test_X)

In [36]:
result = [item[1] for item in result]
result

[0.0005293366824749417,
 0.03613565752374427,
 0.002585735991622814,
 0.0015161269188012595,
 0.00158624812760472,
 0.0009524545095219848,
 0.0007120038130775732,
 0.001694371717622933,
 0.0027621101787371665,
 0.0028556945996887372,
 0.0007548480550277118,
 0.0023127425066325276,
 0.003991420233112344,
 0.016831489190690357,
 0.0023484620950662886,
 0.004735428908741838,
 0.22994324113953105,
 0.003781314696630403,
 0.0011284567298747244,
 0.004094058917889714,
 0.0019297035234184935,
 0.0030714382512098868,
 0.003311055833514744,
 0.0016876384400340168,
 0.00217077012692186,
 0.3038554670639643,
 0.0008744449107122831,
 0.45193253370840397,
 0.002235035923269852,
 0.0028833160561546264,
 0.002043208636227297,
 0.0019740037435069476,
 0.35020893159097194,
 0.0007028879566704637,
 0.0032105315962323955,
 0.36311254579081664,
 0.003148020244095431,
 0.0006805564285467836,
 0.0036659071551098893,
 0.4805306516362622,
 0.17967918297772667,
 0.0008073686721325414,
 0.004947681512161565,
 0

In [37]:
submission = pd.DataFrame()
submission['ID'] = test['ID']
submission['Label'] = result
submission

,ID,Label
0,0,0.000529
1,2,0.036136
2,4,0.002586
3,5,0.001516
4,6,0.001586
5,7,0.000952
6,8,0.000712
7,11,0.001694
8,12,0.002762
9,13,0.002856


In [38]:
submission.to_csv('submission.csv', index=False, encoding='UTF8')